In [1]:
import os
import ipywidgets as widgets
from IPython.display import clear_output
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pyro
import pyro.distributions as dist
# gui
#import qgrid
# pyro & pytorch
import torch
from ipywidgets import fixed, interact, interactive
from ipyfilechooser import FileChooser
from tqdm import tqdm
from scipy.io import loadmat
from sklearn.metrics import matthews_corrcoef, confusion_matrix

from cosmos.utils.visualize import view_parameters, view_glimpse, view_m_probs, \
    view_aoi, view_single_aoi, view_globals, view_params_last
from cosmos.utils.aoi_reader import ReadAoi

%matplotlib inline
#qgrid.enable()

%load_ext tensorboard
%load_ext autoreload
%autoreload 2

In [2]:
device = torch.device("cpu")

def tb_view(path):
    path = os.path.join(path, "scalar")
    %tensorboard --logdir $path

In [3]:
def load_dataset():
    global data, control
    if dataset.selected is not None:
        #data, control = ReadAoi(path=dataset.selected_path, control="control", device=device)
        data, _ = ReadAoi(path=dataset.selected_path, control=None, device=device)

        results.default_path = dataset.selected_path
        with fc:
            clear_output()
            display(results)

        # Show or hide folder icons
        results.use_dir_icons = True

        # Change the title (use '' to hide)
        results.title = "<b>Choose Results Folder</b>"

        results.register_callback(run_app)

def run_app(*args):
    if results.selected is not None:
        pyro.clear_param_store()
        pyro.get_param_store().load(
            filename=os.path.join(results.selected_path, "params"),
            map_location=device)

        data.predictions = np.load(
            os.path.join(results.selected_path, "predictions.npy"))
        
        data.results = pd.read_csv(
            os.path.join(results.selected_path, "params_last.csv"),
            header=None, squeeze=True, index_col=0)

    aoi_widget = widgets.Dropdown(
        options=data.target.index
    )
    frame_widget = widgets.Dropdown(
        options=data.drift.index, value=data.drift.index[0]
    )
    frames_widget = widgets.IntRangeSlider(
        value=[data.drift.index[0], data.drift.index[-1]], min=data.drift.index[0],
        max=data.drift.index[-1], continuous_update=False,
        layout=widgets.Layout(width="100%")
    )
    
    data_widget = widgets.Dropdown(
            options=["d", "c"]
        )
        
    params_widget = widgets.SelectMultiple(
        options=["intensity", "x", "y", "width", "background"],
        value=["intensity", "x", "y", "width"],
        #rows=10,
        description="Parameters",
        disabled=False
    )
    
    
    acc = {}
    #acc["All"] = data.labels
    acc["FN"] = (data.labels["z"] == 1) & (data.predictions["z"] == 0)
    acc["FP"] = (data.labels["z"] == 0) & (data.predictions["z"] == 1)
    acc["TN"] = (data.labels["z"] == 0) & (data.predictions["z"] == 0)
    acc["TP"] = (data.labels["z"] == 1) & (data.predictions["z"] == 1)
    
    acc_widget = widgets.Dropdown(
        options=acc.keys(),
        value="FN",
    )
    acc_aoi_widget = widgets.Dropdown(
        options=np.unique(data.labels["aoi"][acc[acc_widget.value]])
    )
    acc_frame_widget = widgets.Dropdown(
        options=data.labels["frame"][acc[acc_widget.value] & (data.labels["aoi"] == acc_aoi_widget.value)]
    )

    def update_aoi_widget(*args): # *args represent zero (case here) or more arguments.
        acc_aoi_widget.options = np.unique(data.labels["aoi"][acc[acc_widget.value]])
        acc_aoi_widget.value = np.unique(data.labels["aoi"][acc[acc_widget.value]])[0]
        
    def update_frame_widget(*args): # *args represent zero (case here) or more arguments.
        acc_frame_widget.options = data.labels["frame"][acc[acc_widget.value] & (data.labels["aoi"] == acc_aoi_widget.value)]
        acc_frame_widget.value = data.labels["frame"][acc[acc_widget.value] & (data.labels["aoi"] == acc_aoi_widget.value)][0]
    
    acc_aoi_widget.observe(update_frame_widget)
    acc_widget.observe(update_aoi_widget)
    

    w1 = interactive(view_m_probs, aoi=aoi_widget, data=fixed(data), frames=frames_widget,
            m=False, z=False, sp=False, labels=True, predictions=False, prefix=data_widget)


    w2 = interactive(view_parameters, aoi=aoi_widget, data=fixed(data), frames=frames_widget,
            m=False, params=params_widget, prefix=data_widget)

    w3 = interactive(view_aoi, aoi=aoi_widget, frame=frame_widget, data=fixed(data),
             z=False, sp=False, labels=True, predictions=False, target=False, prefix=data_widget)
    
    w4 = interactive(view_single_aoi, aoi=acc_aoi_widget, frame=acc_frame_widget, data=fixed(data),
             z=False, sp=False, labels=True, target=False, acc=acc_widget, prefix=data_widget)
    
    w5 = interactive(view_globals, z=False, j=False, data=fixed(data))
    
    w6 = interactive(view_params_last, results=fixed(data.results))
    
    #w6 = interactive(tb_view, path=fixed(results.selected_path))
    
    tab = widgets.Tab()
    tab.children = [w1, w2, w3, w4, w5, w6]
    tab.titles = ["m_probs", "params", "images", "image", "globals", "info"]
    for i, title in enumerate(tab.titles):
        tab.set_title(i, title)
        
    refresh = widgets.Button(
        description="Refresh",
        disabled=False,
        button_style="", # 'success', 'info', 'warning', 'danger' or ''
        tooltip="Click me",
        icon="check" # (FontAwesome names without the `fa-` prefix)
    )
    
    refresh.on_click(run_app)
    with app:
        clear_output()
        display(refresh, tab)

In [4]:
# Create and display a FileChooser widget
dataset = FileChooser(".")
display(dataset)

results = FileChooser(".")

# Show or hide folder icons
dataset.use_dir_icons = True

# Change the title (use '' to hide)
dataset.title = "<b>Choose Dataset Folder</b>"

dataset.register_callback(load_dataset)

fc = widgets.Output()
app = widgets.Output()
display(fc)
display(app)

FileChooser(path='.', filename='', show_hidden='False')

Output()

Output()

In [1]:
#from tensorboard import notebook
#notebook.list() # View open TensorBoard instances